In [3]:
# import packages
# print function needed later on. __future__ imports need to be first line
from __future__ import print_function

import pandas as pd
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [4]:
# get dataframes from github >> open file >> raw 
glbconfirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
glbrecovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
glbdeath_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
country_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv")

In [5]:
 # data cleaning (renaming to lower-case)
country_df.columns = map(str.lower, country_df.columns)
glbrecovered_df.columns = map(str.lower, glbrecovered_df.columns)
glbdeath_df.columns = map(str.lower, glbdeath_df.columns)
glbconfirmed_df.columns = map(str.lower, glbconfirmed_df.columns)

In [6]:
# change province/state to state and country/region to country
glbconfirmed_df = glbconfirmed_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
glbrecovered_df = glbrecovered_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
glbdeath_df = glbdeath_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns = {'country_region': 'country'})

In [7]:
country_df["confirmed"] = country_df["confirmed"].astype('Int64')
country_df["deaths"] = country_df["deaths"].astype('Int64')
country_df["active"] = country_df["active"].astype('Int64')
country_df["recovered"] = country_df["recovered"].astype('Int64')
# display(country_df.dtypes)

In [8]:
sorted_country_df = country_df.sort_values('confirmed', ascending=False)
sorted_country_df.drop(columns=['lat', 'long_'], inplace = True)
# sorted_country_df

In [9]:
# plotting with plotly

import plotly.express as px


In [10]:
#fig = px.scatter(sorted_country_df.head(10), x='country', y='confirmed', size='confirmed',
                #color='country', hover_name="country", size_max = 60)
# fig.update_layout(title_text = "Confirmed Covid-19 Cases", title_font_size= 24)
#fig.show()

In [11]:
import plotly.graph_objects as go


# nochmal ansehen wie das hier genau funktioniert! wichtig.
def plot_cases_for_country(country):
    labels = ["confirmed", "deaths"]
    colors = ["blue", "red"]
    mode_size = [6, 8]
    line_size = [4, 5]

    df_list = [glbconfirmed_df, glbdeath_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis = 0)
            
        else: 
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:, 5:]), axis = 0)
            
        fig.add_trace(go.Scatter(x = x_data, y = y_data, mode = 'lines+markers', 
                                name = labels[i],
                                line = dict(color = colors[i], width = line_size[i]),
                                connectgaps = True,
                                text = "Total" + str(labels[i]) + ": " + str(y_data[-1])
                                ))
        
    fig.show()  

# plot_cases_for_country('China')

In [12]:
import folium

In [13]:
world_map = folium.Map(location=[11, 0], tiles = "cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)

for i in range(len(glbconfirmed_df)):
    folium.Circle(
    location = [glbconfirmed_df.iloc[i]['lat'], glbconfirmed_df.iloc[i]['long']],
    fill = True,
    radius = (int((np.log(glbconfirmed_df.iloc[i, -1] + 1.00001))) + 0.2) * 50000, 
    fill_color = 'blue',
    color = 'red',
    ).add_to(world_map)
    
# world_map

In [14]:
# opens the voila dashboard and opens the jupyter file with it problem: loop
# deshalb alternativ direkt über cmd voila ...
# import sys
# !voila C:\Users\vanek\Documents\Unterlagen\Programmieren\CS50\final_project/covid_analysis.ipynb


In [15]:
# Following are the Graphs which are in Voila:

In [16]:
print("This is an analysis of the most recent data covering Covid-19 distributed by John Hopkins University. For further information please visit: https://coronavirus.jhu.edu/about")

This is an analysis of the most recent data covering Covid-19 distributed by John Hopkins University. For further information please visit: https://coronavirus.jhu.edu/about


In [17]:
# interact should show me an option to change the country directly!

interact(plot_cases_for_country, country='World');

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

In [18]:
# define a function 'higlight_col' that takes a dataframe, changes certain column colors by using a temp
# and returns the dataframe. 

def highlight_col(x):
    r = 'background-color: salmon'
    p = 'background-color: plum'
    g = 'background-color: palegreen'
    temp_df = pd.DataFrame('', index = x.index, columns = x.columns)
    temp_df.iloc[:, 4] = p
    temp_df.iloc[:, 5] = r
    temp_df.iloc[:, 6] = g
    return temp_df
    
sorted_country_df.head(10).style.apply(highlight_col, axis = None)

,country,last_update,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
176,US,2020-11-18 08:26:30,11359804,248687,4293640,6817477,3447.941288,nan,nan,2.189184,840,USA
79,India,2020-11-18 08:26:30,8912907,130993,8335109,446805,645.860774,nan,nan,1.469700,356,IND
23,Brazil,2020-11-18 08:26:30,5911758,166699,5358299,386760,2781.226212,nan,nan,2.819787,76,BRA
62,France,2020-11-18 08:26:30,2087183,46346,148669,1892041,3197.595680,nan,nan,2.220505,250,FRA
141,Russia,2020-11-18 08:26:30,1954912,33619,1465051,456242,1339.582166,nan,nan,1.719719,643,RUS
160,Spain,2020-11-18 08:26:30,1510023,41688,150376,1317959,3229.665294,nan,nan,2.760753,724,ESP
180,United Kingdom,2020-11-18 08:26:30,1414359,52839,3148,1358372,2083.432396,nan,nan,3.735897,826,GBR
6,Argentina,2020-11-18 08:26:30,1329005,36106,1148833,144066,2940.551282,nan,nan,2.716769,32,ARG
85,Italy,2020-11-18 08:26:30,1238072,46464,457798,733810,2047.691975,nan,nan,3.752932,380,ITA
37,Colombia,2020-11-18 08:26:30,1211128,34381,1118902,57845,2380.226718,nan,nan,2.838759,170,COL
